In [1]:
from pyspark.ml.evaluation import RegressionEvaluator    # for regression evaluation on the output
from pyspark.ml.recommendation import ALS    # for alternating least squares algorithm for matrix factorization
from pyspark.sql.types import IntegerType    # for changing df column to int data type
from pyspark.sql import SparkSession    # main entry point for DataFrame and SQL functionality
from os import path   # for deleting predictions folder
import pandas as pd   # for data frames
import numpy          # for arrays
import time           # for timing cells          
import shutil         # for deleting predictions folder

In [2]:
spark = SparkSession.builder.appName('Matrix Factorization').getOrCreate()
spark

In [3]:
training_df = spark.read.csv("trainItem.data", header = False)
training_df.show(5)

+------+------+---+
|   _c0|   _c1|_c2|
+------+------+---+
|199808|248969| 90|
|199808|  2663| 90|
|199808| 28341| 90|
|199808| 42563| 90|
|199808| 59092| 90|
+------+------+---+
only showing top 5 rows



In [4]:
training_df.count()

12403575

In [5]:
training_df = training_df.withColumnRenamed('_c0', 'userID').withColumnRenamed('_c1', 'itemID').withColumnRenamed('_c2', 'rating')

In [6]:
training_df.dtypes    # returns all columns names and their datatypes

[('userID', 'string'), ('itemID', 'string'), ('rating', 'string')]

In [7]:
training_df = training_df.withColumn('userID', training_df['userID'].cast(IntegerType()))
training_df = training_df.withColumn('itemID', training_df['itemID'].cast(IntegerType()))
training_df = training_df.withColumn('rating', training_df['rating'].cast('float'))

In [8]:
training_df.dtypes    # returns all columns names and their datatypes

[('userID', 'int'), ('itemID', 'int'), ('rating', 'float')]

In [9]:
als = ALS(
    maxIter=40,              # max number of iterations of the model
    rank=15,                 # number of latent features
    regParam=1.0,            # regularization parameter
    userCol="userID",        # user column (ids must be ints)
    itemCol="itemID",        # item column (ids must be ints)
    ratingCol="rating",      # ratings column
    nonnegative = True,      # whether to use nonnegative constraint for least squares
    implicitPrefs = False    # whether to use implicit preference
    #coldStartStrategy="drop" # drop any rows in the df of predictions that contain NaN values
)

In [10]:
start_time = time.time()

model = als.fit(training_df)

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Done! Time elapsed - {elapsed_time:.2f} seconds.')

Py4JJavaError: An error occurred while calling o65.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.StackOverflowError
java.lang.StackOverflowError
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at java.io.ObjectStreamClass.invokeWriteReplace(ObjectStreamClass.java:1245)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1136)
	at java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1548)
	at java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1509)
	at java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1432)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1178)
	at java.io.ObjectOutputSt

In [ ]:
testing_df = spark.read.csv('testItem.data', header=False)
testing_df.show(5)

In [ ]:
testing_df.count()

In [ ]:
testing_df = testing_df.withColumnRenamed("_c0", "userID").withColumnRenamed("_c1", "itemID").withColumnRenamed("_c2", "rating")

In [ ]:
testing_df.dtypes

In [ ]:
testing_df = testing_df.withColumn("userID", testing_df["userID"].cast(IntegerType()))
testing_df = testing_df.withColumn("itemID", testing_df["itemID"].cast(IntegerType()))
testing_df = testing_df.withColumn("rating", testing_df["rating"].cast('float'))

In [ ]:
testing_df.dtypes

In [ ]:
predictions = model.transform(testing_df)    # add 'predictions' column to df

In [ ]:
predictions = predictions.drop('rating')    # remove ratings column (since it's all just 0s)

In [ ]:
predictions.show(5)

In [ ]:
predictions.count()

In [ ]:
# Check if this folder exists already and if so remove it
if path.exists('predictions'):
    shutil.rmtree('predictions') 

In [ ]:
# Save data frame into a folder called 'predictions' 
# with a single file (coalesce(1)) (you cannot assign the filename)
predictions.coalesce(numPartitions=1).write.csv("predictions")

In [ ]:
# Save data frame to a single csv file 
predictions.toPandas().to_csv('als_predictions.csv')

In [ ]:
# Open files
ratings_file = 'als_predictions.csv'
predictions_file = 'predictions.csv'
final_predictions_file = 'final_predictions.csv'

f_ratings = open(ratings_file)
f_predictions = open(predictions_file, 'w')
f_final_predictions = open(final_predictions_file, 'w')

In [ ]:
# Write header
f_final_predictions.write('TrackID,Predictor\n')

In [ ]:
# Read csv into df
column_list = ('userID', 'itemID', 'prediction')
ratings_df = pd.read_csv(ratings_file, usecols=column_list)

In [ ]:
# If any value in prediction column is NaN then replcae it with 50.0
ratings_df['prediction'] = ratings_df['prediction'].fillna(50.0)

In [ ]:
ratings_df

In [ ]:
# Sort by userID (asc) and then by prediction (desc)
ratings_df.sort_values(["userID", "prediction"], ascending = (True, False), inplace=True)

In [ ]:
# Reset index to remove original index
ratings_df.reset_index(drop=True, inplace=True)

In [ ]:
ratings_df.head(12)

In [ ]:
# Save to csv without index column and header row
ratings_df.to_csv('predictions.csv', index=False, header=False)

In [ ]:
f_predictions.close()    # close the file

In [ ]:
f_predictions = open(predictions_file)    # now open it for reading

In [ ]:
# Initialize some values
ratings_array = numpy.zeros(shape=(6))
last_user_id = -1
track_id_out_vec = [0] * 6

In [ ]:
start_time = time.time()

# Go through each line of the predictions file
for line in f_predictions:
    arr_out = line.strip().split(',')    # remove any spaces/new lines and create list 
    user_id_out = arr_out[0]             # set user
    track_id_out = arr_out[1]            # set track
    rating = float(arr_out[2])           # set rating
    
    if user_id_out != last_user_id:             # if new user reached
        i = 0                                   # reset i
        ratings_array = numpy.zeros(shape=(6))  # reset this array
        
    ratings_array[i] = rating                   # add rating to ratings array
    track_id_out_vec[i] = track_id_out          # add trackID to trackID array
        
    i = i + 1                    # increment i
    last_user_id = user_id_out   # set last_user_id as current userID
    
    if i == 6:                               # if last entry for current user reached
        # Here we set the predictions 
        predictions = numpy.zeros(shape=(6)) # initialize numpy array for predictions
        for index in range(0, 3):            
            predictions[index] = 1           # set first 3 values in array to 1 (other 3 are 0)
        
        # Here we write to the final predictions file for the 6 track predictions for the current user
        for ii in range(0, 6):         
            out_str = str(user_id_out) + '_' + str(track_id_out_vec[ii]) + ',' + str(int(predictions[ii]))
            f_final_predictions.write(out_str + '\n')

        
end_time = time.time()
elapsed_time = end_time - start_time
print(f'Done! Time elapsed - {elapsed_time:.2f} seconds.')

In [ ]:
f_ratings.close()
f_predictions.close()
f_final_predictions.close()